In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),  # Normalize the images to range [-1, 1]
    ])
train_transform = transforms.Compose([
    transform,
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)


Files already downloaded and verified


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
resnet_model = models.resnet18(weights=None)
resnet_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

In [5]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 200 mini-batches
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 200:.3f}")
            running_loss = 0.0
    scheduler.step()

print("Finished Training ResNet18 on CIFAR-10")

[Epoch 1, Batch 100] Loss: 1.461
[Epoch 1, Batch 200] Loss: 0.947
[Epoch 1, Batch 300] Loss: 0.896
[Epoch 2, Batch 100] Loss: 0.753
[Epoch 2, Batch 200] Loss: 0.714
[Epoch 2, Batch 300] Loss: 0.699
[Epoch 3, Batch 100] Loss: 0.631
[Epoch 3, Batch 200] Loss: 0.615
[Epoch 3, Batch 300] Loss: 0.613
[Epoch 4, Batch 100] Loss: 0.568
[Epoch 4, Batch 200] Loss: 0.542
[Epoch 4, Batch 300] Loss: 0.537
[Epoch 5, Batch 100] Loss: 0.502
[Epoch 5, Batch 200] Loss: 0.503
[Epoch 5, Batch 300] Loss: 0.498
[Epoch 6, Batch 100] Loss: 0.463
[Epoch 6, Batch 200] Loss: 0.470
[Epoch 6, Batch 300] Loss: 0.461
[Epoch 7, Batch 100] Loss: 0.435
[Epoch 7, Batch 200] Loss: 0.444
[Epoch 7, Batch 300] Loss: 0.436
[Epoch 8, Batch 100] Loss: 0.410
[Epoch 8, Batch 200] Loss: 0.402
[Epoch 8, Batch 300] Loss: 0.420
[Epoch 9, Batch 100] Loss: 0.388
[Epoch 9, Batch 200] Loss: 0.381
[Epoch 9, Batch 300] Loss: 0.389
[Epoch 10, Batch 100] Loss: 0.361
[Epoch 10, Batch 200] Loss: 0.364
[Epoch 10, Batch 300] Loss: 0.370
[Epoch 

In [6]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=True, num_workers=2)

Files already downloaded and verified


In [7]:
correct = 0
total = 0
resnet_model.eval()
with torch.no_grad():
  for i, data in enumerate(testloader, 0):
          inputs, labels = data[0].to(device), data[1].to(device)
          outputs = resnet_model(inputs)
          if(i==0):
            print(f"Output dimensions: {outputs.size()}\nLabel dimensions: {labels.size()}")
          outputs = torch.argmax(outputs, dim=1)
          correct += torch.sum(torch.eq(outputs, labels))
          total += 100
print(f"Test accuracy: {correct/total*100:.2f}%")


Output dimensions: torch.Size([100, 1000])
Label dimensions: torch.Size([100])
Test accuracy: 83.96%
